In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import tkinter as tk
from tkinter import messagebox, ttk
import sqlite3
from datetime import datetime

# -------------------------
# Load and prepare data
# -------------------------
heart_data = pd.read_csv('heart_disease_data.csv')

x = heart_data.drop(columns='target', axis=1)
y = heart_data['target']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, stratify=y, random_state=2
)

# Train the model
model = LogisticRegression(max_iter=1000)
model.fit(x_train, y_train)

# -------------------------
# Database Setup
# -------------------------
conn = sqlite3.connect("heart_predictions.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS predictions (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    age REAL, sex REAL, cp REAL, trestbps REAL, chol REAL,
    fbs REAL, restecg REAL, thalach REAL, exang REAL,
    oldpeak REAL, slope REAL, ca REAL, thal REAL,
    result TEXT, timestamp TEXT
)
""")
conn.commit()

# -------------------------
# Tkinter GUI
# -------------------------
root = tk.Tk()
root.title("❤️ Heart Disease Prediction")
root.geometry("600x780")
root.configure(bg="#eaf6f6")

# Title
title_label = tk.Label(root, text="Heart Disease Prediction System", font=("Arial", 20, "bold"), bg="#00796b", fg="white", pady=10)
title_label.pack(fill="x")

# Frame for inputs
frame = tk.Frame(root, bg="#ffffff", bd=2, relief="ridge", padx=20, pady=20)
frame.pack(pady=20)

labels = [
    "Age", "Sex (1=Male, 0=Female)", "Chest Pain Type (0-3)",
    "Resting BP", "Cholesterol", "Fasting Blood Sugar (1/0)",
    "Resting ECG (0-2)", "Max Heart Rate", "Exercise Angina (1/0)",
    "Oldpeak", "Slope (0-2)", "Major Vessels (0-3)", 
    "Thal (1=Normal, 2=Fixed Defect, 3=Reversible Defect)"
]

entries = []

for i, label in enumerate(labels):
    tk.Label(frame, text=label, font=("Arial", 11, "bold"), bg="#ffffff").grid(row=i, column=0, padx=10, pady=6, sticky="w")
    entry = ttk.Entry(frame, width=25, font=("Arial", 10))
    entry.grid(row=i, column=1, padx=10, pady=6)
    entries.append(entry)

# Result Label
result_label = tk.Label(root, text="", font=("Arial", 14, "bold"), bg="#eaf6f6")
result_label.pack(pady=10)

# Prediction function
def predict():
    try:
        input_data = [float(entry.get()) for entry in entries]
        input_data_as_numpy_array = np.asarray(input_data).reshape(1, -1)
        prediction = model.predict(input_data_as_numpy_array)

        if prediction[0] == 0:
            result_text = "No Heart Disease"
            result_label.config(text="✅ The person does NOT have heart disease.", fg="green")
        else:
            result_text = "Heart Disease"
            result_label.config(text="⚠️ The person HAS heart disease.", fg="red")

        # Save to database
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cursor.execute("""
            INSERT INTO predictions (
                age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang,
                oldpeak, slope, ca, thal, result, timestamp
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (*input_data, result_text, timestamp))
        conn.commit()

    except ValueError:
        messagebox.showerror("Invalid Input", "Please enter valid numerical values in all fields.")

# Predict Button
predict_btn = tk.Button(root, text="🔍 Predict", command=predict, bg="#00796b", fg="white", font=("Arial", 14, "bold"), padx=15, pady=5, relief="raised")
predict_btn.pack(pady=15)

# View History function
def view_history():
    history_window = tk.Toplevel(root)
    history_window.title("📜 Prediction History")
    history_window.geometry("900x400")
    history_window.configure(bg="#f0f0f0")

    tree = ttk.Treeview(history_window, columns=("age","sex","cp","trestbps","chol","fbs","restecg","thalach","exang","oldpeak","slope","ca","thal","result","timestamp"), show="headings")
    tree.pack(fill="both", expand=True)

    # Set column headings
    headings = labels + ["Result", "Timestamp"]
    for col in tree["columns"]:
        tree.heading(col, text=col)
        tree.column(col, width=80, anchor="center")

    cursor.execute("SELECT * FROM predictions")
    rows = cursor.fetchall()
    for row in rows:
        tree.insert("", "end", values=row[1:])  # skip ID

# View History Button
history_btn = tk.Button(root, text="📜 View Prediction History", command=view_history, bg="#004d40", fg="white", font=("Arial", 12, "bold"))
history_btn.pack(pady=5)

# Footer
footer_label = tk.Label(root, text="Developed with ❤️ using Python, Tkinter & SQLite", font=("Arial", 9, "italic"), bg="#eaf6f6", fg="#555")
footer_label.pack(side="bottom", pady=5)

root.mainloop()


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
